In [30]:
import pandas as pd
from datetime import datetime
from docxtpl import DocxTemplate
from gtts import gTTS
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from docx import Document
import os

In [31]:
inventory = pd.read_excel("Inventory.xlsx")
sells = pd.read_excel("Sells.xlsx")

In [32]:
merged_df = pd.merge(inventory,sells, on="product_id",how="left")
merged_df["final_stock"] = merged_df["stock"] - merged_df["sells"]
merged_df

,product_id,Item,stock,price,min_stock,sells,final_stock
0,1,GPU,10,700,3,8,2
1,2,Keyboard,15,200,5,5,10
2,3,Mouse,15,100,5,1,14
3,4,Mousepad,15,50,5,1,14
4,5,CPU,8,500,3,5,3
5,6,Case,12,200,5,1,11
6,7,Monitor,6,700,2,3,3
7,8,Ram sticks,16,100,6,12,4


In [33]:
critical_df = merged_df[merged_df['final_stock'] < merged_df['min_stock']]
critical_df

,product_id,Item,stock,price,min_stock,sells,final_stock
0,1,GPU,10,700,3,8,2
7,8,Ram sticks,16,100,6,12,4


In [ ]:
count = len(critical_df) if 'critical_df' in globals() else 0
text = f'Atención CEO. El proceso ha finalizado. Se han detectado {count} productos con stock crítico. Revise el reporte en Word.'
output_audio = 'critical_summary.mp3'


tts = gTTS(text, lang='es')
tts.save(output_audio)

In [ ]:
workbook = load_workbook("Inventory.xlsx")
sheet = workbook.active
header = [cell.value for cell in sheet[1]]
def find_col_index(header_row, name):
    name_lower = str(name).strip().lower()
    for i, v in enumerate(header_row):
        if v is None:
            continue
        if str(v).strip().lower() == name_lower:
            return i + 1
    raise ValueError(f"Column '{name}' not found in header: {header_row}")
id_col_index = find_col_index(header, 'product_id')
stock_col_index = find_col_index(header, 'stock')

In [36]:
doc = Document()
doc.add_heading('Critical Stock Report', level=1)
rows = critical_df.to_dict(orient='records')
for r in rows:
    product_id = r.get('product_id')
    stock_val = r.get('final_stock', r.get('stock'))
    product_id_str = '' if product_id is None else str(product_id)
    stock_str = '' if stock_val is None else str(stock_val)
    text = f"The product (id: {product_id_str}) is short on stock ({stock_str})"
    doc.add_paragraph(text)
output_path = 'critical_report_python_docx.docx'
doc.save(output_path)